# Test results

With folder `data/cleaned/preds`, we can obtain the results for the position and error metrics but in an accumulated version and not as an average.

In [2]:
import glob, os, pandas as pd, numpy as np, torch, math
from tqdm import tqdm
from BronchoTrack import metrics

def compute_metrics_acum(folder):
    aape, aade, aane, aace, aamse, aaqe = [], [], [], [], [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        acum = df.cumsum()
        pos_preds = torch.from_numpy(acum.loc[acum.index[-1], ["shift_x", "shift_y", "shift_z"]].values)
        pos_targets = torch.from_numpy(acum.loc[acum.index[-1], ["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
        aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
        rot_targets = acum.loc[acum.index[-1], ["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
        rot_preds = acum.loc[acum.index[-1], ["Rx_dif", "Ry_dif", "Rz_dif"]].values
        rot_targets = torch.from_numpy(rot_targets)
        rot_preds = torch.from_numpy(rot_preds)
        aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
        aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=5))
        cse = metrics.CosMetric()
        cse.update(rot_preds, rot_targets)
        aace.append(cse.compute())
        q = metrics.QuatMetric()
        q.update(rot_preds, rot_targets)
        aaqe.append(q.compute())
        m = metrics.MSE()
        m.update(rot_preds, rot_targets)
        aamse.append(m.compute())
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {} AACE {} +- {} AAMSE {} +- {} AAQE {} +- {}".format(
        np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane), np.mean(aace), np.std(aace),
        np.mean(aamse), np.std(aamse), np.mean(aaqe), np.std(aaqe))

In [15]:
compute_metrics_acum("data/cleaned/preds_30traj_MSE/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_COS/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_DE/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_QUAT/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_COS_EU/*.csv")

100%|██████████| 120/120 [00:00<00:00, 203.34it/s]
/home/juan/miniconda3/envs/broncho/lib/python3.6/site-packages/numpy/core/_methods.py:202: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
100%|██████████| 120/120 [00:00<00:00, 195.90it/s]


('AAPE 13.835903226179187 +- 5.759193861172195, AADE 1.5742343056554189 +- 0.6551976729299223, AANE 15.376781348145277 +- 6.102938251610644 AACE 0.94476717710495 +- 0.40410280227661133 AAMSE 330.4622497558594 +- 1490.69140625 AAQE 1.225030779838562 +- 0.4022783935070038',
 'AAPE 2.7214844740006536 +- 1.617503149440687, AADE 1.5950016855184932 +- 0.7031537380339785, AANE 7.131795900345185 +- 2.6656356540799773 AACE 0.6926049590110779 +- 0.3571860194206238 AAMSE 1175.4071044921875 +- 4128.2587890625 AAQE 1.4041533470153809 +- 0.42319968342781067',
 'AAPE 3.8744685928356146 +- 2.1056544343301424, AADE 1.5879705693521076 +- 0.6207517413373088, AANE 7.937667314923589 +- 2.789065596879322 AACE 0.9652023911476135 +- 0.37166428565979004 AAMSE 1354.478515625 +- 4346.8916015625 AAQE 1.250084400177002 +- 0.36366134881973267',
 'AAPE nan +- nan, AADE nan +- nan, AANE nan +- nan AACE nan +- nan AAMSE nan +- nan AAQE inf +- nan',
 'AAPE 2.2824132663835672 +- 1.3827900781865636, AADE 1.56133145557103

In [3]:
def compute_metrics(folder):
    aape, aade, aane, aace, aamse, aaqe = [], [], [], [], [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        for row, data in df.iterrows():
            pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
            aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3))
            cse = metrics.CosMetric()
            cse.update(rot_preds, rot_targets)
            aace.append(cse.compute())
            q = metrics.QuatMetric()
            q.update(rot_preds, rot_targets)
            aaqe.append(q.compute())
            m = metrics.MSE()
            m.update(rot_preds, rot_targets)
            aamse.append(m.compute())
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {} AACE {} +- {} AAMSE {} +- {} AAQE {} +- {}".format(
        np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane), np.mean(aace), np.std(aace),
        np.mean(aamse), np.std(aamse), np.mean(aaqe), np.std(aaqe))

In [13]:
compute_metrics("data/cleaned/data_increase/outer_120/preds_120traj_COS_EU_inter_P18/*.csv")

100%|██████████| 480/480 [01:33<00:00,  5.14it/s]


'AAPE 0.741318184442516 +- 0.33942156852382965, AADE 0.9163618778503371 +- 0.6556021034026633, AANE 2.639044526232093 +- 1.5604850960864711 AACE 0.27053242921829224 +- 0.2927328646183014 AAMSE 1.7874747514724731 +- 4.655580520629883 AAQE 0.8203817009925842 +- 0.5043652653694153'

In [18]:
compute_metrics("data/cleaned/preds_30traj_MSE/*.csv"), compute_metrics("data/cleaned/preds_30traj_COS/*.csv"), compute_metrics("data/cleaned/preds_30traj_DE/*.csv"), compute_metrics("data/cleaned/preds_30traj_QUAT/*.csv"), compute_metrics("data/cleaned/preds_30traj_COS_EU/*.csv")

100%|██████████| 120/120 [00:17<00:00,  6.93it/s]


('AAPE 0.5491277217666873 +- 0.573653173740771, AADE 0.7248276525217127 +- 0.498294865650828, AANE 2.133701451221059 +- 1.3643027063455693 AACE 0.17344273626804352 +- 0.19498062133789062 AAMSE 15.751633644104004 +- 455.8465576171875 AAQE 0.6379731297492981 +- 0.3691841959953308',
 'AAPE 0.2646646795385093 +- 0.16944264858997296, AADE 0.5428455738276546 +- 0.5032078981195843, AANE 1.5545040837767403 +- 1.2577643932778806 AACE 0.11609118431806564 +- 0.18177255988121033 AAMSE 25.304031372070312 +- 711.93359375 AAQE 0.6825854182243347 +- 0.4137953817844391',
 'AAPE 0.3682859682206188 +- 0.1913224449289966, AADE 0.713015618811532 +- 0.5331496554219437, AANE 2.0406247662187673 +- 1.3392663732322747 AACE 0.43466493487358093 +- 0.3853222131729126 AAMSE 26.017972946166992 +- 706.2825927734375 AAQE 0.749027669429779 +- 0.5646618604660034',
 'AAPE nan +- nan, AADE nan +- nan, AANE nan +- nan AACE nan +- nan AAMSE nan +- nan AAQE inf +- nan',
 'AAPE 0.25759985880518227 +- 0.16911057809014685, AADE

 ## Results loss cross patient

In [36]:
from collections import defaultdict

In [50]:
def compute_errors(root="./data/cleaned/preds_round_1/", losses=["COS", "MSE", "DE", "QUAT"]):
    subfolders = {i: [os.path.join(root, folder)  for folder in os.listdir(root) if i in folder] for i in losses}
    submetrics = {i: defaultdict(list) for i in losses}
    for loss in tqdm(losses):
        for subfolder in subfolders[loss]:
            for file in os.listdir(subfolder):
                df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0)
                for row, data in df_sample.iterrows():
                    pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
                    pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
                    submetrics[loss]["AAPE"].append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
                    rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
                    rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
                    rot_targets = torch.from_numpy(rot_targets)
                    rot_preds = torch.from_numpy(rot_preds)
                    submetrics[loss]["AADE"].append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy())
                    submetrics[loss]["AANE"].append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3).numpy())
        submetrics[loss]["AAPE"] = (np.mean(submetrics[loss]["AAPE"]), np.std(submetrics[loss]["AAPE"]))
        submetrics[loss]["AADE"] = (np.mean(submetrics[loss]["AADE"]), np.std(submetrics[loss]["AADE"]))
        submetrics[loss]["AANE"] = (np.mean(submetrics[loss]["AANE"]), np.std(submetrics[loss]["AANE"]))
    return submetrics
        


In [51]:
results = compute_errors()

  0%|          | 0/4 [00:00<?, ?it/s]

In [52]:
results

{'COS': defaultdict(list,
             {'AAPE': (0.7378152121278749, 0.4505294373549075),
              'AADE': (1.1975325795102736, 0.7669386701924008),
              'AANE': (3.253374431203741, 1.6608348329290958)}),
 'MSE': defaultdict(list,
             {'AAPE': (0.7555466617392615, 0.4499440452227794),
              'AADE': (1.2367228395662788, 0.7639783827566352),
              'AANE': (3.3043340700818193, 1.6125088852780254)}),
 'DE': defaultdict(list,
             {'AAPE': (0.7390488814078968, 0.4434353750507124),
              'AADE': (1.2119055731497668, 0.7702147501155583),
              'AANE': (3.2856565743090784, 1.666478388639561)}),
 'QUAT': defaultdict(list,
             {'AAPE': (0.7443752694683445, 0.44549288678195853),
              'AADE': (1.332005785650499, 0.6744398102944591),
              'AANE': (3.6061799354533117, 1.4897843823402674)})}

# Build docs for Debora

In [21]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import torch
from BronchoTrack import metrics

In [31]:
def add_metrics(df_x):
    df_x["Direction Error"], df_x["Pos Error (Euclidean)"], df_x["Needle Error"], df_x["Cosinus Error"], df_x["Cosinus X Error"] = 0.0, 0.0, 0.0, 0.0, 0.0
    df_x["Cosinus Y Error"], df_x["Cosinus Z Error"], df_x["MSE"], df_x["Quaternion Error"] = 0.0, 0.0, 0.0, 0.0
    for row, data in df_x.iterrows():
            pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            df_x.loc[row, "Pos Error"] = metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy()
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            df_x.loc[row, "Direction Error"] = metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy()
            df_x.loc[row, "Needle Error"] = metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3).numpy()
            cse = metrics.CosMetric()
            cse.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus Error"] = cse.compute().numpy()
            q = metrics.QuatMetric()
            q.update(rot_preds, rot_targets)
            df_x.loc[row, "Quaternion Error "] = q.compute().numpy()
            m = metrics.MSE()
            m.update(rot_preds, rot_targets)
            df_x.loc[row, "MSE"] = m.compute().numpy()
            cse_x = metrics.CosMetric(0)
            cse_x.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus X Error"] = cse_x.compute().numpy()
            cse_y = metrics.CosMetric(1)
            cse_y.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus Y Error"] = cse_y.compute().numpy()
            cse_z = metrics.CosMetric(2)
            cse_z.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus Z Error"] = cse_z.compute().numpy()
    return df_x


def add_info(df_x, filename, folder):
    df_x["Patient"] = filename.split("_")[1]
    df_x["Lobe"] = filename.split("_")[-2]
    df_x["Trajectory"] = filename.split("_")[-1].split(".")[0]
    df_x["Loss"] = folder.split("_")[-2]
    return df_x

In [32]:
root = "./data/cleaned/loss_intra/"
final_csv = pd.DataFrame()
for folder in tqdm(os.listdir(root)):
    subfolder = os.path.join(root, folder)
    if os.path.isdir(subfolder) and "preds" in subfolder:
        for file in os.listdir(subfolder):
            df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0) 
            df_sample = add_metrics(df_sample)
            df_sample = add_info(df_sample, file, folder)
            if not final_csv.empty:
                final_csv = pd.concat([final_csv, df_sample], axis=0, ignore_index=True)
            else:
                final_csv = df_sample.copy()
final_csv.to_csv(os.path.join(root, "global_result.csv"))

100%|██████████| 7/7 [01:55<00:00, 16.52s/it]
